In [ ]:
page_from = 1
# http://wyborcza.pl/0,75248.html?str=1_23719317
page_to = 10
# http://wyborcza.pl/0,75248.html?str=10_23719317

In [2]:
from selenium import webdriver
import pandas as pd
from scrap_article_current_tab import scrap_article

from datetime import datetime as dt
from datetime import timedelta as td
from os import makedirs
from os.path import exists
from time import sleep
from json import loads

In [3]:
now = dt.now()
year = now.year
month = now.month
day = now.day
ts = str(int(round(now.timestamp())))

In [4]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
chrome = webdriver.Chrome(options=options)

In [5]:
# chrome = webdriver.Chrome()

In [6]:
chrome.get(url = 'http://wyborcza.pl/0,75248.html?str=100_23719317')

In [7]:
sleep(10)

In [8]:
# try to click rodo (first click opens sometimes an ad also)
for i in range(3):
    try:
        sleep(3)
        chrome.find_element_by_xpath('//*[@id="rodoNotificationWrapper"]/div[1]/div/p[2]')\
            .click()
    except:
        pass

In [9]:
# close new window if exists
try:
    chrome.switch_to.window(chrome.window_handles[1])
    chrome.close()
except:
    pass

In [10]:
chrome.switch_to.window(chrome.window_handles[0])

In [11]:
chrome.get('https://login.wyborcza.pl/')

In [12]:
chrome\
    .find_element_by_xpath('//*[@id="wyborczaEmail"]')\
    .send_keys('slusarczyk1@wp.pl')

In [13]:
chrome\
    .find_element_by_xpath('//*[@id="wyborczaPassword"]')\
    .send_keys('Sraniejebanko1')

In [14]:
chrome\
    .find_element_by_xpath('/html/body/section/section[1]/form/div[4]/button')\
    .click()

In [15]:
sleep(3)

In [16]:
# http://wyborcza.pl/0,93566.html?str=2_23721352 for video

In [17]:
urls = []
# NUMBERS TO CHANGE \/
for i in range(page_from, page_to):
    chrome.get(url = 'http://wyborcza.pl/0,75248.html?str='+str(i)+'_23719317')
    for entry in chrome\
        .find_element_by_xpath('//*[@id="pagetype_index"]/section[5]/div/main/div/div[2]/div/div[2]/ul')\
        .find_elements_by_class_name('entry'):
        
        url = entry.find_element_by_tag_name('h3')\
                .find_element_by_tag_name('a')\
                .get_attribute('href')
        #scrap_article(chrome, url, first_time)
        urls.append(url)

In [18]:
import pandas as pd

In [19]:
df = pd.DataFrame(urls, columns = ['urls'])

In [20]:
# backup
df.to_csv('FILENAME_WITH_SCRAPPED_URLS_TO_SAVE.csv')

In [21]:
df = pd.read_csv('FILENAME_WITH_SCRAPPED_URLS_TO_SAVE.csv')

In [22]:
errors = []

In [23]:
for i, url in enumerate(df['urls']):
    #print(url)
    try:
        a = scrap_article(chrome, url, True)
        with open('comments/'+a['url'].split('/')[-1].split(',')[2], 'w') as f:
            f.write(str({'row': {
               'comments': a,
               'timestamp': dt.timestamp(dt.now())
               }
            }))
        #print(a['url'].split('/')[-1].split(',')[2])

    except:
        errors.append(url)
        

http://lublin.wyborcza.pl/lublin/7,48724,24793298,w-dolhobyczowie-platforma-otworzyla-granice-z-ukraina-przyszedl.html
http://wyborcza.pl/7,75399,24800431,beda-przedterminowe-wybory-w-austrii-koniec-romansu-chadecji.html
http://wyborcza.pl/7,75398,24799022,premier-morawiecki-zakonczenie-roku-szkolnego-dwa-dni-wczesniej.html
http://wyborcza.pl/7,75398,24798828,tusk-glosujac-na-europe-glosujecie-tez-na-wolna-demokratyczna.html
http://kielce.wyborcza.pl/kielce/7,47262,24798558,dwoch-policjantow-rannych-podczas-interwencji-jeden-z-napastnikow.html
http://wyborcza.pl/7,75399,24798352,polityczne-trzesienie-ziemi-w-wiedniu-beda-nowe-wybory.html
http://wyborcza.pl/7,75400,24797843,krwawa-zemsta-na-hiszpanach-350-czlonkow-karawany-hernana-cortesa.html
http://wyborcza.pl/7,75399,24796962,australijscy-aborygeni-skarza-swoj-rzad-do-onz-po-raz-pierwszy.html
http://wyborcza.pl/7,155287,24796353,tak-zarabia-sie-dzis-na-mieszkaniach-kupujesz-remontujesz.html
http://wyborcza.pl/7,75398,24797627,wyborcz

In [25]:
pd.DataFrame(errors, columns = ['urls']).to_csv('error_urls.csv')